# RespInPeace: Toolkit for precessing breathing belt (RIP) data

In [ ]:
import datetime
import matplotlib.pyplot as plt

# Silence FutureWarnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

## Data loading

Load data from a WAV file and plot the respiratory signal:

In [ ]:
import rip

resp = rip.RIP('rip_data.wav')

print('''Sampling frequency: {}
Number of samples: {}
Duration: {}'''.format(resp.samp_freq, len(resp.resp), datetime.timedelta(seconds=resp.dur)))

In [ ]:
plt.plot(resp.t / 60, resp.resp)

## Resampling

Resampling is done simply with the `resample` method.

In [ ]:
resp.resample(100)

In [ ]:
print("""Sampling frequency: {}
Number of samples: {}""".format(resp.samp_freq, len(resp.resp)))

In [ ]:
plt.plot(resp.resp)

# Detrending and drift removal

**RespInPeace** implements several detrending methods:
- Subracting the mean with `detrend(type='constant')`
- Removal of linear trend with `detrend(type='linear')`
- Removal with low-frequency baseline oscilation.

Below we demonstrate the last of these methods:

In [ ]:
resp.remove_baseline()
plt.plot(resp.resp)

## Signal segmentation

Identify inhalation and exhalation onsets

In [ ]:
resp.find_cycles()

Internally, inhalations, exhalations and respiratory cycles are stored as two-dimensional numpy arrays.

In [ ]:
resp.inhalations[:10, :]

In [ ]:
resp.exhalations[:10, :]

In [ ]:
resp.cycles[:10, :]

Here is a plot showing the first couple of cycles.

In [ ]:
plt.plot(resp.t, resp.resp, color='lightgrey')
plt.plot(resp.t[resp.inhalations[:, 0]], resp.resp[resp.inhalations[:, 0]],
         linestyle='none', marker='o')
plt.plot(resp.t[resp.exhalations[:, 0]], resp.resp[resp.exhalations[:, 0]],
         linestyle='none', marker='o')
plt.xlim(0,  1e4 /  resp.samp_freq)

Given this representations, calculating durational features is extremely easy. For instance, below  we calculate and plot the distribution of inhalation durations:

In [ ]:
inh_durs = resp.inhalations[:, 1] - resp.inhalations[:, 0]
_ = plt.hist(inh_durs / 60, bins=20)

We can also plot them:

## Range estimation

In [ ]:
resp.estimate_range()

plt.plot(resp.resp, color='lightgrey')
plt.axhline(resp.range_bot, linestyle='dashed')
plt.axhline(resp.range_top, linestyle='dashed')

## REL estimation

Resting expiratory level(REL) is the state of equilibrium when the expiratory elasticity forces equal the the inhalatory elasticity forces. It also the point within the total lung capacity where speakers are most likely to inhale as speaking at lung volumes below REL is generally felt to be quite uncomfortable.

REL is quite sensitive to posture shifts and if the subject did not stand perfectly still, might fluctuate in the course of a recording. In order to neutralize the effect this drift, REL needs to be estimated in a more dynamic way. RestInPeace estimates REL by as the median signal level at respiratory troughs in the preceding interval of specified duration (??? by default).

In [ ]:
resp.estimate_rel(20)

In [ ]:
plt.plot(resp.resp, color='lightgrey')
plt.plot(resp.inhalations[:, 0], resp.rel)

In [ ]:
plt.plot(resp.resp, color='lightgrey')
plt.plot(resp.inhalations[:, 0], resp.rel)
plt.xlim(0, 1e4)